In [1]:
#Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install langchain openai lark tiktoken faiss numpy gym==0.25.1 anthropic --upgrade 

In [2]:
import os, json
os.environ['JAVA_HOME'] = "/home/yh/anaconda3/envs/webshop/bin/"
os.environ['JVM_PATH'] = "/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so"
# Open the file and load the JSON data
with open("./.api_keys.txt", "r") as file:
    api_keys = json.load(file)

os.environ["OPENAI_API_KEY"] = api_keys["OPENAI_API_KEY"]
os.environ['ANTHROPIC_API_KEY'] = api_keys["ANTHROPIC_API_KEY"]
os.environ['REPLICATE_API_TOKEN']=api_keys["REPLICATE_API_TOKEN"]

In [3]:
import gc
gc.collect()

380

In [4]:
import os

# Change to a directory
os.chdir('./webshop/baseline_models')


from train_rl import parse_args as webenv_args
import imp
from env import WebEnv 
import webshop.baseline_models.test as test_utils


args = test_utils.args
env = test_utils.env

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/faiss/loader.py:28: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numpy.__version__) >= "1.19":
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/tmp/ipykernel_848/1366313140.py:8: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


Products loaded.

Keys cleaned.

Attributes loaded.

100%|█████████████████████████████████████████████████████████████████████| 1181436/1181436 [00:10<00:00, 115540.98it/s]


164 skipped

Loaded 12087 goals.


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


Total huamn goals considered： range(0, 500)
env loaded
30522
30526


In [5]:
args.bart_path = './ckpts/web_search/checkpoint-3600'
args.image = True
args.model_path = "./ckpts/web_click_image/epoch_9/model.pth"
import torch
bart_model = test_utils.BartForConditionalGeneration.from_pretrained(args.bart_path)
print('bart model loaded', args.bart_path)

config = test_utils.BertConfigForWebshop(image=args.image)
model = test_utils.BertModelForWebshop(config)
model.cuda()
model.load_state_dict(torch.load(args.model_path), strict=False)
print('bert il model loaded', args.model_path)


bart model loaded ./ckpts/web_search/checkpoint-3600


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


bert il model loaded ./ckpts/web_click_image/epoch_9/model.pth


In [27]:
from importlib import reload
import expert_action_predictor
reload(expert_action_predictor)
from expert_action_predictor import ExpertActionPredictor
expert_predictor = ExpertActionPredictor(bart_model, model, test_utils.predict, softmax=True)

In [7]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool


from langchain.chat_models import ChatOpenAI
from output_parser import (
    AutoGPTOutputParser,
    BaseAutoGPTOutputParser,
)
from prompt import AutoGPTPrompt
from langchain.chains.llm import LLMChain

import numpy as np
import json

DELIMITER="%%%%"     #Pick an unlikely delimiter used somewhere else

def convert_info(info: dict) -> dict:
    global env
    actions = []
    for a in info['valid']:
        if a.startswith('click[item - '):
            actions.append(f'click[{env.name2asin[a[13:-1]]}]')
        else:
            actions.append(a)
    info['valid'] = actions
    
    if info['image_feat'] is not None:
        numpy_array = info['image_feat'].numpy()  # Convert tensor to NumPy array
        info['image_feat'] = numpy_array.tolist()  # Serialize the NumPy array to JSON

    return info

def search(action: str) -> str:
    global env, bart_model, model
    obs, reward, done, info = env.step(f"search[{action}]")
    info = convert_info(info)
    # assert len(valid_actions) > 0
    
    return "=Observation=\n" + obs  + DELIMITER + json.dumps(info)

def click(action: str) -> str:
    global env, bart_model, model
    obs, reward, done, info = env.step(f"click[{action}]")
    obs_prompt = "=Observation=\n" + obs
    info = convert_info(info)
    
    #if valid_actions[0].startswith("search["):
    #    return obs_prompt + "\n:Valid Actions:\n['search[<QUERY>]']"
    #else:
    #    action_prompt = "\nValid Actions:\n" + str(valid_actions)
    if done: 
        return obs_prompt + "{'reward':%s}" % reward    
    return obs_prompt  + DELIMITER + json.dumps(info)

def process_result(full_msg_log):
    import re
    # print(reward_msg)
    output_parser = AutoGPTOutputParser()

    parsed_action = output_parser.parse(full_msg_log[-1].content)
    if parsed_action.name == "finish":
        reward_msg = full_msg_log[-4].content
        match = re.search(r"'reward':([\d\.]+)", reward_msg)
        if match:
            reward = float(match.group(1))
        else:
            reward = 0.0
        return {"purchase": True, "reward": reward * 10}
    else:
        return {"purchase": False, "reward": 0.0}

def init_vectorstore(embedding_size=1536):
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})
    return vectorstore

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/langchain/vectorstores/analyticdb.py:20: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()  # type: Any


In [8]:
obs, info = env.reset(0)
action = expert_predictor.predict(obs, info, top_k=5)
print(action)

obs, reward, done, info = env.step('search[gluten free, 100% vegan plant based protein shake, soy-free, under $50]')
# print(obs)

action = expert_predictor.predict(obs, info, top_k=5)
print(action)


obs, reward, done, info = env.step('click[B07FYPSNH8]')
#print(info)
convert_info(info)
# print(type(info['image_feat']))
info['image_feat'] = torch.tensor(info['image_feat'], dtype=torch.float32)
action = expert_predictor.predict(obs, info)
print(action)



/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)


['search[gluten free, 100% vegan plant based protein shake]', 'search[soy free, 100% vegan plant based protein shake]', 'search[vegan plant based protein shake that is soy-free]', 'search[plant based protein shake that is soy-free]', 'search[vegan plant based protein shake]']


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


['click[item - owyn 100% vegan plant-based protein shake, cold brew coffee, 36 pack, 20g plant protein, 148mg of caffeine, omega-3, prebiotic supplements, superfoods greens blend, gluten-free, soy-free, non-gmo]', 'click[item - owyn 100% vegan plant-based protein shake, cookies & creamless, 1 count, with 20g plant protein, omega-3, prebiotic supplements, superfoods greens blend, gluten-free, soy-free, non-gmo]', 'click[item - owyn 100% vegan plant-based protein shake, cookies & creamless, 1 count, with 20g plant protein, omega-3, prebiotic supplements, superfoods greens blend, gluten-free, soy-free, non-gmo]', 'click[item - owyn 100% vegan plant-based protein shake, cold brew coffee, 36 pack, 20g plant protein, 148mg of caffeine, omega-3, prebiotic supplements, superfoods greens blend, gluten-free, soy-free, non-gmo]', 'click[item - owyn 100% vegan plant-based protein shake, strawberry banana, 36 pack, 20g plant protein with omega-3, prebiotic supplements, superfoods greens blend, glut

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


In [9]:
SEARCH_TOOL_SPEC_V1 = """
    Requirement:
    1. You can conduct search if your valid action has 'search[<QUERY>]'. 
    2. You need to come up with the best search query based on the instruction in the Observation.
    3. You need to check your past search queries to avoid repetitive searches.
    4. Try to relax constraint on the search query if you cannot find right items.
    Example:
    After you come up with the best query, output in the format as search[<query>].
    For example, for instruction 'i need a long clip-in hair extension which is natural looking,
    and price lower than 30.00 dollars', the query could be like
    'long clip-in hair extension natural looking, lower than 30.00 dollars'
"""

CLICK_TOOL_SPEC_V1 = """
    Requirement:
    1. You should strictly limit the choises of your actions from the Valid Action list, all in 'click[<input>]' format.
    2. In output JSON, use "click" for "command.name", and use the variable <input> for "command.args.tool_input".  
    3. Please make the purchase if you are running out of actions, indicated in 'loop number:'.
    Example: 
    1. The valid actions has'click[b09mm9ncqq]', you use 'click' as "command.name" and 'b09mm9ncqq' as "command.args.tool_input". 
     The actual item description for 'b09mm9ncqq' can be found in the Observation section. 
    2. The valid actions has 'click[back to search]', you use 'click' as "command.name" and 'back to search' as "command.args.tool_input".
    3. The valid actions has 'click[buy now]', you use click as "command.name" and "buy now" as "command.args.tool_input".

"""

SEARCH_TOOL_SPEC_V2 = """
####
    Requirement: Search when there is a search button. 
    Try different search queries if you cannot find any matched items in first few pages.
    Example 1:
        =Observation=
            Instruction: 
            i'd love some help finding some organic beard growth oil that can treat hair loss, and price lower than 40.00 dollars
            [button] Search [button_]
        =Response=
            {
                "thoughts": {
                    "text": "There is only one search button, so I can only search",
                    "reasoning": "...",
                    "plan": "...",
                    "criticism": "..."
                },
                "command": {
                    "name": "search",
                    "args": {
                        "tool_input": "organic beard growth oil, under $40"
                        }
                }
            }
####
"""

CLICK_TOOL_SPEC_V2 = """
####
    Requirement: Click the [button]{content}[button_] with the {content}.
                 You shouldn't click when there is only one search button where you should just search.
    Example 1:
        =Observation=
            Instruction:
            i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 40.00 dollars
            [button] Back to Search [button_]
            Page 1 (Total results: 50)
            [button] Next > [button_]

            [button] B07FYPSNH8 [button_]
            OWYN - 100% Vegan Plant-Based Protein Shakes | Cold Brew Coffee, 12 Fl Oz | Dairy-Free, Gluten-Free, Soy-Free, Tree Nut-Free, Egg-Free, Allergy-Free, Vegetarian
            $11.07

            [button] B08BPTT8MX [button_]
            Pediasure Grow & Gain with Immune Support, Kids Protein Shake, 27 Vitamins and Minerals, 7g Protein, Helps Kids Catch Up On Growth, Non-GMO, Gluten-Free, Chocolate, 8 Fl Oz, 16 Count
            $100.0
        =Response=
        {
            "thoughts": {
                "text": "...",
                "reasoning": "I can only click 'Next >' to got to the next page, or click an item like B07FYPSNH8",
                "plan": "...",
                "criticism": "I shouldn't click "Buy Now" as there's no such [button]."
            },
            "command": {
                "name": "click",
                "args": {
                    "tool_input": "B07FYPSNH8"
                }
            }
        }
    Example 2:
        =Observation=
            Instruction:
            i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 40.00 dollars
            [button] Back to Search [button_]
            [button] < Prev [button_]
            OWYN - 100% Vegan Plant-Based Protein Shakes | Cold Brew Coffee, 12 Fl Oz | Dairy-Free, Gluten-Free, Soy-Free, Tree Nut-Free, Egg-Free, Allergy-Free, Vegetarian
            Price: $11.07
            Rating: N.A.
            [button] Description [button_]
            [button] Features [button_]
            [button] Reviews [button_]
            [button] Buy Now [button_]
        =Response=
        {
            "thoughts": {
                "text": "...",
                "reasoning": " I can only click 'Back to Search' to search again, or '< Prev' to got to the previous page,
            or 'Description','Features', 'Reviews' to check details, or 'Buy Now' to make the purchase.",
                "plan": "Here the item description matches the instruction and there's no other option to choose, I will click 'Buy Now'.",
                "criticism": "N/A"
            },
            "command": {
                "name": "click",
                "args": {
                    "tool_input": "Buy Now"
                }
            }
        }
    Example 3:
        =Observation=
            Instruction:
            hello, i'm looking for a pair of cargo pants for everyday casual wear? but also hiking-friendly. also, i want an orange pair please, and price lower than 50.00 dollars
            [button] Back to Search [button_]
            [button] < Prev [button_]
            color
            [button] orange [button_]
            [button] yellow [button_]
            size
            [button] small [button_]
            [button] medium [button_]
            Mens Casual Cargo Pants Hi Vis Viz Reflective Overalls High Visibility Safe Work Pants Outdoor Hiking Trousers Big and Tall
            Price: $16.79
            Rating: N.A.
            [button] Description [button_]
            [button] Features [button_]
            [button] Reviews [button_]
            [button] Buy Now [button_]
        =Response=
        {
            "thoughts": {
                "text": "...",
                "reasoning": "I can only click 'Back to Search' to search again, or '< Prev' to got to the previous page, or a color or size option to select the specific product option, or 'Description','Features','Reviews' to check details, or 'Buy Now' to make the purchase.",
                "plan": "Here the item description mentions 'organge pair', so I will click 'orange'.",
                "criticism": "N/A."
            },
            "command": {
                "name": "click",
                "args": {
                    "tool_input": "orange"
                }
            }
        }
    Example 4:
        =Observation=
            Instruction:
            I would like a 7 foot oval turquoise rug that is easy to spot clean.
            [button] Back to Search [button_]
            [button] < Prev [button_]
            Transform any living space with the Home Dynamix Premium Area Rug Collection. 
            The versatile and durable Premium area rug collection by Home Dynamix offers style and beauty at an affordable price.
            Instantly elevate your décor to the next level with these designer inspired accent rugs.
        =Response=
        {
            "thoughts": {
                "text": "...",
                "reasoning": "I can only click 'Back To Search' or '< Prev'. Since the description matches my instruction,",
                "plan": "I will go back to the item page again by clicking '< Prev'.",
                "criticism": "N/A"
            },
            "command": {
                "name": "click",
                "args": {
                    "tool_input": "< Prev"
                }
            }
        }
####
"""



In [10]:
webshop_search_tool = Tool(
    name = "search",
    func=search,
    description=SEARCH_TOOL_SPEC_V2
)

webshop_click_tool = Tool(
    name = "click",
    func = click,
    description=CLICK_TOOL_SPEC_V2
)

tools = [
    webshop_search_tool,
    webshop_click_tool
]

In [11]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss




In [12]:
from langchain.chat_models import ChatAnthropic
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from vicuna_llm import VicunaLLM

ai_name = "Shopping AI"
ai_role = "Web Shopper"
llm_gpt3_5 = ChatOpenAI(temperature=0.01)
llm_gpt4 = ChatOpenAI(model_name ="gpt-4", temperature=0.01)
llm_claude = ChatAnthropic(model="claude-v1", max_tokens_to_sample=4196, temperature=0.01)
llm_vicuna = VicunaLLM(max_length=10000, repetition_penalty=1, temperature = 1)

In [13]:
from langchain.chat_models import ChatAnthropic
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

import requests

class VicunaLLM(LLM):        
    @property
    def _llm_type(self) -> str:
        return "custom"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        response = requests.post(
            "http://127.0.0.1:8000/prompt",
            json={
                "prompt": prompt,
                "temperature": 0,
                "max_new_tokens": 256,
                "stop": stop + ["Observation:"]
            }
        )
        response.raise_for_status()
        return response.json()["response"]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {

        }

In [14]:
def display_result(results):
    import pandas as pd
    import numpy as np
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(results, dtype = np.float64)

    df_string = df.to_csv(sep='\t', index=False)
    print(df_string)

import json

def write_dict_to_file(dictionary, file_path):
    # Open the file in write mode
    with open(file_path, "w") as file:
        for line in dictionary:
            file.write(str(line) + "\n")


In [28]:
import autogpt
from importlib import reload
reload(autogpt)
from autogpt import AutoGPT


LOOP_LIMIT = 20
input_variables=["memory", "messages", "goals", "user_input"]
expert_predictor = ExpertActionPredictor(bart_model, model, test_utils.predict, softmax=True)
def init_agent(ai_name, ai_role, tools, input_variables,
               llm, vectorstore, init_obs = None, verbose = True,
               expert_predictor=expert_predictor, top_k=1, random=True):
    prompt = AutoGPTPrompt(
                ai_name=ai_name,
                ai_role=ai_role,
                tools=tools,
                input_variables=input_variables,
                token_counter=llm_gpt3_5.get_num_tokens
            )
    chain = LLMChain(llm=llm, prompt=prompt)


    agent = AutoGPT(
        ai_name="Shopping Expert",
        memory=vectorstore.as_retriever(),
        chain=chain,
        output_parser=AutoGPTOutputParser(),
        tools=tools,
        loop_limit=LOOP_LIMIT,
        init_obs=init_obs,
        expert_predictor=expert_predictor,
        top_k=top_k,
        random=random
    )
    agent.chain.verbose = verbose
    return agent




In [29]:
results = []
full_logs = []
def run_llm(llm, start, end, expert_predictor=None, top_k=1, random=False):
    global results, full_logs
    for i in range(start, end):
        obs, info = env.reset(i)
        print(f"#### Goal {i}: {info['goal']} ####")
        agent = init_agent(ai_name, ai_role, tools, input_variables, llm,
                           init_vectorstore(), init_obs=obs + DELIMITER + json.dumps(info),
                           verbose=False, expert_predictor=expert_predictor, top_k=top_k, random=random)

        goal = f"#### Find the right item and make the right purchase in {LOOP_LIMIT} actions. " + info['goal']
        full_msg_log, actions = agent.run([goal])
        results.append(process_result(full_msg_log))
        full_logs.append(full_msg_log)
    return results, full_logs


In [34]:
start, end = 0, 50
run = "run_1_random"

In [31]:
import warnings
warnings.filterwarnings('ignore')
# Example usage
# file_path = "full_logs_test_item_15_to_19_gpt_4_alone.txt"
# file_path = "full_logs_test_item_20_39_claude_il.txt"
# file_path = "full_logs_test_item_20_39_gpt_4_alone.txt"
# file_path = "full_logs_test_item_30_40_gpt_4_il.txt"

results = []
full_logs = []
results, full_logs = run_llm(llm_claude, start, end, expert_predictor=expert_predictor, top_k=1, random=True)
#results, full_logs = run_llm(llm_claude, 30, 40, expert_predictor=expert_predictor)




#### Goal 0: Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars ####
loop number:1
Here's one suggestion for the command: search with 'gluten free, 100% vegan plant based protein shake'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "There is only one search button, so I can only search",
        "reasoning": "The instruction specifies to search for 'gluten free, 100% vegan plant based protein shake', so I will search for that.",
        "plan": "Search for the item, check the results and select the matched item.",
        "criticism": "N/A"
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input": "gluten free, 100% vegan plant based protein shake"
        }
    }
}
Command search returned: =Observation=
Instruction:
i 

loop number:5
Here's one suggestion for the command: click with 'reviews'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "I can only click 'Back to Search' to search again, or '< Prev' to got to the previous page, or 'Description','Features', 'Reviews' to check details, or 'Buy Now' to make the purchase.",
        "reasoning": "Since the item description matches the instruction, I will check the reviews to confirm.",
        "plan": "Check the reviews and then decide whether to purchase.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "reviews"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars
[button] Back to Search [button_]


loop number:9
Here's one suggestion for the command: click with 'b08vr5w9cl'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "I found an item that matches the instruction in this page.",
        "reasoning": "The item 'B08VR5W9CL OWYN 100% Vegan Plant-Based Protein Shake, Smooth Vanilla, 36 Pack, with 20g Plant Protein, Omega-3, Prebiotic supplements, Superfoods Greens Blend, Gluten-Free, Soy-Free, Non-GMO' matches the instruction, so I will click it.",
        "plan": "Click 'B08VR5W9CL' and check the item details.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "B08VR5W9CL"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars
[bu

loop number:13
Here's one suggestion for the command: click with 'next >'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 Here is my response in JSON format:
{
    "thoughts": {
        "text": "I found an item that matches the instruction in this page.",
        "reasoning": "The item 'B08VR5W9CL OWYN 100% Vegan Plant-Based Protein Shake, Smooth Vanilla, 36 Pack, with 20g Plant Protein, Omega-3, Prebiotic supplements, Superfoods Greens Blend, Gluten-Free, Soy-Free, Non-GMO' matches the instruction, so I will click it.",
        "plan": "Click 'B08VR5W9CL' and check the item details.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "B08VR5W9CL"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and p

loop number:18
Here's one suggestion for the command: click with '< prev'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 Here is my response in JSON format:
{
    "thoughts": {
        "text": "I have checked the item features and it matches the instruction, so I will go back to the previous page.",
        "reasoning": "Clicking '< prev' will bring me back to the item selection page so I can proceed to purchase the item.",
        "plan": "Click '< prev' to go back to the previous page and click 'Buy Now' to purchase the item.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "< prev"
        }
    } 
}
Command click returned: =Observation=
You have clicked 12 fl oz (pack of 24).
You have clicked vanilla.
Instruction:
i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free

...

loop number:16
Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "I sincerely apologize, I have made multiple mistakes in constructing my JSON responses. Let me slow down and try again.",
        "reasoning": "I need to carefully review the requirements and double check my work to provide a valid JSON response.",
        "plan": "- Slowly and carefully review the observation, instruction and JSON format requirements.",
        "- Determine the appropriate command and arguments to use.",
        "- Construct the JSON response and double check that it is valid before submitting.",
        "criticism": "I have clearly struggled with providing valid JSON responses and need to slow down, focus and be more meticulous in my work. I will make sure I double check all future JSON responses to avoid these types of issues going forward."
    },
    "command": {
        "name": "search",
        "args": {
            "tool_i

loop number:20
Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "I sincerely apologize, I have made multiple mistakes in constructing my JSON responses. Let me slow down and try again.",
        "reasoning": "I need to carefully review the requirements and double check my work to provide a valid JSON response.",
        "plan": "- Slowly and carefully review the observation, instruction and JSON format requirements.",
        "- Determine the appropriate command and arguments to use.",
        "- Construct the JSON response and double check that it is valid before submitting.",
        "criticism": "I have clearly struggled with providing valid JSON responses and need to slow down, focus and be more meticulous in my work. I will make sure I double check all future JSON responses to avoid these types of issues going forward."
    },
    "command": {
        "name": "search",
        "args": {
            "tool_i

loop number:4
Here's one suggestion for the command: click with 'b09mk31mdr'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "After comparing the items, I will go back to the item 'B09MK31MDR' which is the cheapest option that meets the instruction.",
        "reasoning": "The item 'B09MK31MDR' is an organic hair growth serum roller set for $14.99, which is the lowest price among the matched items.",
        "plan": "- Click 'B09MK31MDR' again to check the details\n- If it still matches the instruction, click 'Buy Now' to finish the task",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "B09MK31MDR"
        }
    }
}
Command click returned: =Observation=
Instruction:
show me an one organic hair growth serum roller set for all hair types, and price lower than 60.00 doll

loop number:4
Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "There were errors in my previous responses. I should double check the details and price of each item carefully to find the right one based on the instruction.",
        "reasoning": "The item 'B08CZBMHDR' matches the instruction, with a price lower than $40.",
        "plan": "- Double check the details and price of each matched item carefully", 
        "- Select the right item based on the instruction",
        "criticism": "I should be extremely careful when constructing the JSON response to avoid errors. I need to improve in this aspect."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "B08CZBMHDR"
        }
    }
}
Error: {'error': 'Could not parse invalid json:  {\n    "thoughts": {\n        "text": "There were errors in my previous responses. I should double check the details and price of each ite

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ /tmp/ipykernel_848/2996858041.py:11 in <module>                                           │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_848/2996858041.py'                   │
│                                                                                           │
│ /tmp/ipykernel_848/589619901.py:13 in run_llm                                             │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_848/589619901.py'                    │
│                                                                                           │
│ /home/yh/LLMAgent/autogpt.py:144 in run                                                   │
│                                                                                           │
│   141 │   │   │   print(loop_msg)                                                         │
│   142 │   │   │   print(user_input)                                                       │
│   143 │   │   │   # Send message to AI, get response                                      │
│ ❱ 144 │   │   │   assistant_reply = self.chain.run(                                       │
│   145 │   │   │   │   goals=goals,                                                        │
│   146 │   │   │   │   messages=self.full_message_history,                                 │
│   147 │   │   │   │   memory=self.memory,                                                 │
│                                                                                           │
│ /home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/langchain/chains/base.py:239  │
│ in run                                                                                    │
│                                                                                           │
│   236 │   │   │   return self(args[0], callbacks=callbacks)[self.output_keys[0]]          │
│   237 │   │                                                                               │
│   238 │   │   if kwargs and not args:                                                     │
│ ❱ 239 │   │   │   return self(kwargs, callbacks=callbacks)[self.output_keys[0]]           │
│   240 │   │                                                                               │
│   241 │   │   if not kwargs and not args:                                                 │
│   242 │   │   │   raise ValueError(                                                       │
│                                                                                           │
│ /home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/langchain/chains/base.py:140  │
│ in __call__                                                                               │
│                                                                                           │
│   137 │   │   │   )                                                                       │
│   138 │   │   except (KeyboardInterrupt, Exception) as e:                                 │
│   139 │   │   │   run_manager.on_chain_error(e)                                           │
│ ❱ 140 │   │   │   raise e                                                                 │
│   141 │   │   run_manager.on_chain_end(outputs)                                           │
│   142 │   │   return self.prep_outputs(inputs, outputs, return_only_outputs)              │
│   143                                                                                     │
│                                                                                           │
│ /home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/langchain/chains/base.py:134  │
│ in __call__                                       

In [32]:
display_result(results)
#file_path = "full_logs_0_19_gpt3_5.txt"
#file_path = "full_logs_20_29_gpt4_il.txt"

file_path = f"full_logs_{start}_{end-1}_claude_il_{run}.txt"
write_dict_to_file(full_logs, file_path)

purchase	reward
0.0	0.0
1.0	0.0
0.0	0.0
1.0	66.66666666666666



In [54]:
import warnings
warnings.filterwarnings('ignore')
# Example usage
# file_path = "full_logs_test_item_15_to_19_gpt_4_alone.txt"
# file_path = "full_logs_test_item_20_39_claude_il.txt"
# file_path = "full_logs_test_item_20_39_gpt_4_alone.txt"
# file_path = "full_logs_test_item_30_40_gpt_4_il.txt"

results = []
full_logs = []
results, full_logs = run_llm(llm_claude, start, end, expert_predictor=expert_predictor, top_k=5)
#results, full_logs = run_llm(llm_claude, 30, 40, expert_predictor=expert_predictor)




#### Goal 40: Instruction: i'm looking for a flats made of ethylene vinyl. choose the ones in tan color and size 9.5 narrow, and price lower than 80.00 dollars ####
loop number:1
Here's a few suggestions for the command: search with 'tan ethylene vinyl flats', search with 'tan ethylene vinyl flats size 9.5', search with 'tinted ethylene vinyl flats', search with 'tan ethylene vinyl flats 9.5 narrow', search with 'tan ethylene vinyl flats 9.5'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "There are multiple search options suggested. I will try searching with the most specific query first.",
        "reasoning": "Searching with a more specific query containing size and width information is more likely to yield relevant results.",
        "plan": "- Try searching with 'tan ethylene vinyl flats 9.5 narrow' first\n- If no good results, try s

loop number:4
Here's a few suggestions for the command: click with 'tan leather metallic synthetic', click with 'tan leather metallic synthetic', click with 'buy now', click with 'buy now', click with 'buy now'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "The item description matches the instruction. I should proceed to purchase the item.",
        "reasoning": "Since the selected size and color options match the instruction, this seems to be the desired item. Clicking 'Buy Now' is the logical next step.",
        "plan": "- Click 'Buy Now' to purchase the item\n- Finish the task if purchase is successful",
        "criticism": "I should double check that the price is within the budget before purchasing. Just matching the other criteria is not enough."
    },
    "command": {
        "name": "click",
        "args": {
            "tool

loop number:3
Here's a few suggestions for the command: click with '< prev', click with '< prev', click with '< prev', click with '9.5', click with 'back to search'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "The item description matches the requirements, I will select the size and add to cart.",
        "reasoning": "Since the item description mentions 'Snow Boots for Women' and price range matches, I will choose size 7.5 based on the instruction and add to cart.",
        "plan": "- Click '7.5' for the size\n- Click 'Add to Cart'\n- Checkout and finish the purchase",
        "criticism": "I should double check the item details and price again before adding to cart to avoid any mistake."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "7.5"
        }
    } 
}
Command click returned: =Obser

loop number:3
Here's a few suggestions for the command: click with '< prev', click with '< prev', click with 'description', click with '< prev', click with '< prev'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "The item details page shows up. I will check the item description to ensure it matches the instruction before determining the next command.",
        "reasoning": "Checking the item details, especially the description, is important to confirm it is the desired product before purchasing.",
        "plan": "- Click 'Description' to view item details\n- Check if item description matches instruction\n- If match, click 'Buy Now' to purchase\n- If not match, click '< Prev' to go back and re-analyze search results",
        "criticism": "I should be thorough in checking item details to avoid making an incorrect purchase."
    },
    "co

loop number:3
Here's a few suggestions for the command: click with '< prev', click with '24k gold scrub', click with 'features', click with '< prev', click with '< prev'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "There are scent options to choose from. I will check the item features first to ensure it matches the instruction before purchasing.",
        "reasoning": "Checking the features will provide more details about the item to evaluate if it matches the requested 'pink body scrub for dry skin'.",
        "plan": "- Click 'Features' to check item details\n- If it matches, click 'Buy Now' to purchase\n- If not, click '< Prev' to go back and try other items",
        "criticism": "I should not purchase the item without evaluating the details to ensure it matches the instruction."
    },
    "command": {
        "name": "click",
   

...

loop number:4
Here's a few suggestions for the command: click with 'b07xsbkhlf', click with 'next >', click with 'back to search', click with 'next >', click with 'back to search'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "The item 'Montana Woodworks Montana Collection 9 Drawer Dresser, Ready to Finish' matches my instruction, I will check the details.",
        "reasoning": "I will click 'B07XSBKHLF' to view the item details.",
        "plan": "- Click 'B07XSBKHLF' to check the item details\n- Make the purchase if it meets the requirements\n- Go back to search results if not satisfied",
        "criticism": "I should analyze the item details carefully to make sure it matches the instruction before making the purchase."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "B07XSBKHLF"
        }

 {
    "thoughts": {
        "text": "The item details match the instruction. I will select the options and make the purchase.",
        "reasoning": "Since the instruction specifies a large slim fit red shirt, I will select 'large' size and 'red' color.",
        "plan": "- Click 'large' for size\n- Click 'red' for color\n- Click 'Buy Now' to make the purchase",
        "criticism": "I should follow the instruction closely to select the right options and complete the task."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "large"
        }
    }
}
Command click returned: =Observation=
You have clicked large.
Instruction:
i'm looking for a men's red button down casual shirt that is a large slim fit, and price lower than 30.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
color
  [button] blue [button_]
  [button] red [button_]
  [button] yellow [button_]
size
  [button] medium [button_]
  [clicked button] large [clicked 

loop number:2
Here's a few suggestions for the command: click with 'next >', click with 'next >', click with 'b08k3wxp43', click with 'next >', click with 'back to search'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "There are multiple options to click. I will click 'Next >' first to go to the next page, which may contain more relevant and affordable items.", 
        "reasoning": "Clicking 'Next >' allows me to explore more options without committing to any items yet. This aligns with my plan to check more results and find an item under $90.",
        "plan": "- Click 'Next >' to go to next page\n- Check more results and select an item under $90",
        "criticism": "I should avoid clicking on any specific items before exploring more options on the next page."
    },
    "command": {
        "name": "click",
        "args": {
      

loop number:5
Here's a few suggestions for the command: click with 'buy now', click with 'back to search', click with 'description', click with 'features', click with 'buy now'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "The item details confirm this is a teeth whitening kit specifically for sensitive teeth that is within my budget. I will proceed to purchase.",
        "reasoning": "Clicking 'Buy Now' will allow me to complete the purchase of an item that meets all of my criteria. This aligns with my goal to find an affordable teeth whitening kit for sensitive teeth.",
        "plan": "- Click 'Buy Now' to purchase the selected item\n- Complete the checkout process",
        "criticism": "I should double check the total cost including any additional fees like shipping before completing the purchase to ensure it remains within my budg

loop number:4
Here's a few suggestions for the command: click with 'irish', click with 'irish', click with 'irish', click with 'irish', click with 'irish'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
 {
    "thoughts": {
        "text": "There are multiple 'irish' options suggested. I will choose one that matches the instruction.",
        "reasoning": "Choosing an option that matches the instruction will yield the desired result.",
        "plan": "- Analyze the suggested 'irish' options\\n- Choose the first 'irish' option as it is most likely to match the instruction",
        "criticism": "I should consider all options carefully before making a choice."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "irish"
        }
    }
}
Command click returned: =Observation=
You have clicked irish.
Instruction:
i am looking for kosher ce

In [55]:
display_result(results)
#file_path = "full_logs_0_19_gpt3_5.txt"
#file_path = "full_logs_20_29_gpt4_il.txt"

file_path = f"full_logs_{start}_{end-1}_claude_il_top_5_{run}.txt"
write_dict_to_file(full_logs, file_path)

purchase	reward
1.0	75.0
1.0	100.0
1.0	0.0
1.0	33.33333333333333
1.0	50.0
1.0	25.0
1.0	33.33333333333333
1.0	100.0
1.0	100.0
1.0	66.66666666666666



In [56]:
if False:
    import warnings
    warnings.filterwarnings('ignore')
    # Example usage

    results = []
    full_logs = []
    results, full_logs = run_llm(llm_claude, start, end, expert_predictor=None)
    #results, full_logs = run_llm(llm_claude, 30, 40, expert_predictor=expert_predictor)




In [57]:
if False:
    display_result(results)

    file_path = f"full_logs_{start}_{end-1}_claude_{run}.txt"
    write_dict_to_file(full_logs, file_path)

=================================== GPT3.5 =============================================

In [35]:
import warnings
warnings.filterwarnings('ignore')
# Example usage
# file_path = "full_logs_test_item_15_to_19_gpt_4_alone.txt"
# file_path = "full_logs_test_item_20_39_claude_il.txt"
# file_path = "full_logs_test_item_20_39_gpt_4_alone.txt"
# file_path = "full_logs_test_item_30_40_gpt_4_il.txt"

results = []
full_logs = []
results, full_logs = run_llm(llm_gpt3_5, start, end, expert_predictor=expert_predictor, top_k=1, random=True)
#results, full_logs = run_llm(llm_claude, 30, 40, expert_predictor=expert_predictor)


#### Goal 0: Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars ####
loop number:1
Here's one suggestion for the command: search with 'gluten free, 100% vegan plant based protein shake'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will use the suggested search query to find the product.",
        "reasoning": "The instruction mentions specific requirements for the product, so I will use the suggested search query to find the product.",
        "plan": "- Search with the suggested query\n- Check the first few pages of the search results\n- Look for products that match the requirements\n- Choose a product that is priced lower than 30.00 dollars",
        "criticism": "N/A"
    },
    "command": {
        "name": "search",
        "args": {
            "t

loop number:4
Here's one suggestion for the command: click with 'description'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will click on 'Description' to view the product details.",
        "reasoning": "Clicking on 'Description' will allow me to view the product details and ensure that it matches the requirements.",
        "plan": "- Click on 'Description' to view the product details\n- Check the product details to ensure it matches the requirements\n- Add the product to the cart and proceed to checkout",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Description"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars
[button]

loop number:8
Here's one suggestion for the command: click with 'next >'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will click on 'Next >' to go to the next page of search results.",
        "reasoning": "Clicking on 'Next >' will allow me to see more search results that match the requirements.",
        "plan": "- Click on 'Next >' to go to the next page of search results",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Next >"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars
[button] Back to Search [button_]
Page 3 (Total results: 50)
[button] < Prev [button_]
[button] Next > [button_]

[button] B07DM82K3Y [button_]
OW

{
    "thoughts": {
        "text": "I will click on '< Prev' to go back to the previous page and search again.",
        "reasoning": "Since the product doesn't match the requirements, I will go back to the previous page and search again.",
        "plan": "- Click on '< Prev' to go back to the previous page and search again",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "< Prev"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
flavor name
  [button] coffee [button_]
  [button] cookies & creamless [button_]
  [button] dark chocolate [button_]
  [button] smooth vanilla [button_]
  [button] strawberry banana [button_]
  [button] variety pack [button_]
OWYN 100% Vegan Plant-Based Protein Shake, Smooth Vanilla, 12 Pa

loop number:18
Here's one suggestion for the command: click with 'b0885y2gd4'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will click on 'B078SXRM7S' to view the product details.",
        "reasoning": "Clicking on the item will allow me to view the product details and check if it matches the requirements.",
        "plan": "- Click on 'B078SXRM7S' to view the product details",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "B078SXRM7S"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
OWYN - 100% Vegan Plant-Based Protein Shakes | Dark Chocolate, 12 Fl Oz (Pack of 

...

loop number:15
Here's one suggestion for the command: click with 'description'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will click on 'Description' to learn more about the product.",
        "reasoning": "Clicking on 'Description' will take me to the product description page where I can learn more about the product and determine if it matches the instruction.",
        "plan": "- Click on 'Description'",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Description"
        }
    }
}
Command click returned: =Observation=
Instruction:
i want to find a teeth whitening device kit for sensitive teeth, and price lower than 90.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
Fast, comfortable, easy-to-use at-home teeth whitening with an advanced

loop number:20
Here's one suggestion for the command: click with 'b08kgn21r3'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will click on 'B08KGN21R3' to view the details of the product.",
        "reasoning": "Clicking on 'B08KGN21R3' will take me to the product page where I can view the details of the product.",
        "plan": "- Click on 'B08KGN21R3'",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "B08KGN21R3"
        }
    }
}
Command click returned: =Observation=
Instruction:
i want to find a teeth whitening device kit for sensitive teeth, and price lower than 90.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
LOVE YOUR TEETH Whitening Device Kit - Teeth Whitening System with LED Blue Light Mouthpiece - Home Teeth Whitener - Clinicall

{
    "thoughts": {
        "text": "I will click on the 'Reviews' button to check the reviews of the item.",
        "reasoning": "I need to check the reviews of the item to make sure it is a good purchase. I can do this by clicking on the 'Reviews' button.",
        "plan": "- Click on the 'Reviews' button to check the reviews of the item.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Reviews"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for kosher certified irish fortune cookies with pattern name :halloween, and price lower than 40.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]

loop number:6
Here's one suggestion for the command: click with 'back to search'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "tex

loop number:12
Here's one suggestion for the command: click with '< prev'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will click on the '< Prev' button to go back to the search results.",
        "reasoning": "Since the product features do not mention anything about the pattern name 'halloween', I will go back to the search results to look for another item that matches the instruction.",
        "plan": "- Click on the '< Prev' button.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "< Prev"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for kosher certified irish fortune cookies with pattern name :halloween, and price lower than 40.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
Gefen Gluten Free Kosher

{
    "thoughts": {
        "text": "I will click on 'B008O5PVQQ' to check the details of the item.",
        "reasoning": "I need to check the details of the item to see if it matches the instruction.",
        "plan": "- Click on 'B008O5PVQQ'.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "B008O5PVQQ"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for kosher certified irish fortune cookies with pattern name :halloween, and price lower than 40.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
Graduation Fortune Cookies (50 Individually Wrapped) for Graduation Parties
Price: $17.68
Rating: N.A.
[button] Description [button_]
[button] Features [button_]
[button] Reviews [button_]
[button] Buy Now [button_]

loop number:18
Here's one suggestion for the command: click with '< prev'.
Please use this suggestion as a reference and make your own judgement. Determ

In [36]:
display_result(results)
#file_path = "full_logs_0_19_gpt3_5.txt"
#file_path = "full_logs_20_29_gpt4_il.txt"

file_path = f"full_logs_{start}_{end-1}_gpt3_5_il_{run}.txt"
write_dict_to_file(full_logs, file_path)

purchase	reward
1.0	100.0
0.0	0.0
0.0	0.0
0.0	0.0
0.0	0.0
0.0	0.0
0.0	0.0
1.0	1.6666666666666665
0.0	0.0
0.0	0.0
1.0	33.33333333333333
0.0	0.0
0.0	0.0
1.0	66.66666666666666
0.0	0.0
0.0	0.0
1.0	33.33333333333333
0.0	0.0
1.0	25.0
1.0	33.33333333333333
1.0	66.66666666666666
0.0	0.0
1.0	75.0
1.0	50.0
0.0	0.0
0.0	0.0
0.0	0.0
1.0	40.0
0.0	0.0
1.0	100.0
0.0	0.0
1.0	50.0
0.0	0.0
1.0	33.33333333333333
1.0	50.0
0.0	0.0
1.0	100.0
0.0	0.0
1.0	66.66666666666666
1.0	66.66666666666666
1.0	0.0
0.0	0.0
1.0	50.0
0.0	0.0
0.0	0.0
1.0	25.0
0.0	0.0
1.0	50.0
0.0	0.0
0.0	0.0



In [60]:
import warnings
warnings.filterwarnings('ignore')
# Example usage
# file_path = "full_logs_test_item_15_to_19_gpt_4_alone.txt"
# file_path = "full_logs_test_item_20_39_claude_il.txt"
# file_path = "full_logs_test_item_20_39_gpt_4_alone.txt"
# file_path = "full_logs_test_item_30_40_gpt_4_il.txt"

results = []
full_logs = []
results, full_logs = run_llm(llm_gpt3_5, start, end, expert_predictor=expert_predictor, top_k=5)
#results, full_logs = run_llm(llm_claude, 30, 40, expert_predictor=expert_predictor)

#### Goal 40: Instruction: i'm looking for a flats made of ethylene vinyl. choose the ones in tan color and size 9.5 narrow, and price lower than 80.00 dollars ####
loop number:1
Here's a few suggestions for the command: search with 'tan ethylene vinyl flats', search with 'tan ethylene vinyl flats size 9.5', search with 'tinted ethylene vinyl flats', search with 'tan ethylene vinyl flats 9.5 narrow', search with 'tan ethylene vinyl flats 9.5'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will use the search command with the query 'tan ethylene vinyl flats size 9.5' to find the right item.",
        "reasoning": "I will include the size in the search query to narrow down the results.",
        "plan": "- Search for 'tan ethylene vinyl flats size 9.5'\n- Check the price of the items\n- Click on the item that matches the criteria",
      

I'm sorry, but the previous suggestion is not relevant to the current task. Here's a new suggestion:

{
    "thoughts": {
        "text": "I will click on the 'Description' button to check the details of the product.",
        "reasoning": "Checking the description will give me more information about the product and help me confirm if it matches the criteria.",
        "plan": "- Click on the 'Description' button.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Description"
        }
    }
}
Command click returned: =Observation=
Instruction:
i'm looking for a flats made of ethylene vinyl. choose the ones in tan color and size 9.5 narrow, and price lower than 80.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]

loop number:6
Here's a few suggestions for the command: click with '< prev', click with '< prev', click with '< prev', click with '< prev', click with '< prev'.
Please use this sugges

loop number:10
Here's a few suggestions for the command: click with 'buy now', click with 'buy now', click with 'buy now', click with '< prev', click with 'description'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will click on 'Buy Now' as it is the next step in making the purchase.",
        "reasoning": "Clicking on 'Buy Now' will take me to the checkout page where I can complete the purchase.",
        "plan": "- Click on 'Buy Now'.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Buy Now"
        }
    }
}
Command click returned: =Observation=
Thank you for shopping with us!
Your code: 
None
 (Paste it in your MTurk interface.)
Purchased
asin
B08G185FNP
options
{"size": "9 narrow"}
attrs
None
category
None
query
None
product category
None
Target
asin
option

loop number:4
Here's a few suggestions for the command: click with '< prev', click with 'buy now', click with 'buy now', click with 'buy now', click with 'description'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will click on 'Buy Now'.",
        "reasoning": "The item matches the description and the size is correct, so I will click on 'Buy Now' to make the purchase.",
        "plan": "- Click on 'Buy Now'.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Buy Now"
        }
    }
}
Command click returned: =Observation=
Thank you for shopping with us!
Your code: 
None
 (Paste it in your MTurk interface.)
Purchased
asin
B09QL7SV25
options
{"size": "7.5"}
attrs
None
category
None
query
None
product category
None
Target
asin
options
attrs
price upper
instuction tex

loop number:4
Here's a few suggestions for the command: click with 'next >', click with '< prev', click with 'b08dkxjxsy', click with '< prev', click with 'next >'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "None of the items on the third page match the criteria. I will click '< Prev' to go back to the second page.",
        "reasoning": "Since none of the items on the third page match the criteria, I need to go back to the second page.",
        "plan": "- Click '< Prev' to go back to the second page.\n- Check the first few pages of results.\n- If no suitable items are found, try different search queries.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "< Prev"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for a white item bar

...

loop number:6
Here's a few suggestions for the command: click with '< prev', click with '< prev', click with '< prev', click with '< prev', click with '< prev'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will click on '< Prev' to go back to the search results page.",
        "reasoning": "Clicking on '< Prev' will take me back to the search results page, where I can look for other dressers that match my description.",
        "plan": "- Use the click command with the argument '< Prev'.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "< Prev"
        }
    }
}
Command click returned: =Observation=
Instruction:
i would like a six drawer natural walnut dresser with bronze finishes
[button] Back to Search [button_]
[button] < Prev [button_]
Linon Home Decor Product

{
    "thoughts": {
        "text": "I will click on the '< Prev' button to go back to the search results page.",
        "reasoning": "Clicking on the '< Prev' button will take me back to the search results page, where I can try a different item.",
        "plan": "I will use the click command with the argument '< Prev' to go back to the search results page.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "< Prev"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for a 9 drawer dresser with a bronze finish
[button] Back to Search [button_]
[button] < Prev [button_]
color
  [button] beige antique finish a085 [button_]
  [button] green antique finish a086 [button_]
Bedroom Furniture Hand Hewn from Solid Pine 9 Drawer Dresser Chester, Bedroom Furniture Distressed Vintage Antiqued Finish in 2 Colors (Green Antique Finish a086)
Price: $100.0
Rating: N.A.
[button] Description [button_]


{
    "thoughts": {
        "text": "I will click on the first item that matches the criteria.",
        "reasoning": "I will click on the item because it matches the criteria of being a men's red button down casual shirt that is a large slim fit and under $30.",
        "plan": "- Use the click command with the argument 'b09q94dhc9' to select the first item that matches the criteria.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "b09q94dhc9"
        }
    }
}
Command click returned: =Observation=
Instruction:
i'm looking for a men's red button down casual shirt that is a large slim fit, and price lower than 30.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
color
  [button] green [button_]
  [button] red [button_]
size
  [button] medium [button_]
  [button] large [button_]
  [button] x-large [button_]
  [button] xx-large [button_]
  [button] 3x-large [button_]
Mens Stripe T-Shirts Casual

loop number:2
Here's a few suggestions for the command: click with 'next >', click with 'next >', click with 'b09bkpxthp', click with 'b0936z9bnw', click with 'next >'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will use the click command to navigate to the next page and find the right item.",
        "reasoning": "I can only click 'Next >' to go to the next page, or click an item like 'B09BKPXTHP' or 'B0936Z9BNW'.",
        "plan": "- Use the click command with the query 'Next >' to go to the next page.\n- If I cannot find the right item, I will use the click command to select the item 'B09BKPXTHP' or 'B0936Z9BNW'.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Next >"
        }
    }
}
Command click returned: =Observation=
Instruction:
i want to find a teet

loop number:2
Here's a few suggestions for the command: click with 'b01a99gdy8', click with 'b01a99gdy8', click with 'back to search', click with 'b0083a9kcs', click with 'b01a99gdy8'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I will click on the item with the pattern name 'halloween' and is under $40. I will use the click command with the button id 'B01A99GDY8'.",
        "reasoning": "The item with the button id 'B01A99GDY8' matches the pattern name 'halloween' and is under $40, so I will click on it.",
        "plan": "- Use the click command with the button id 'B01A99GDY8'.",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "B01A99GDY8"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for kosher certified irish fortune cookies w

In [61]:
display_result(results)
#file_path = "full_logs_0_19_gpt3_5.txt"
#file_path = "full_logs_20_29_gpt4_il.txt"

file_path = f"full_logs_{start}_{end-1}_gpt3_5_il_top_5_{run}.txt"
write_dict_to_file(full_logs, file_path)

purchase	reward
0.0	0.0
0.0	0.0
0.0	0.0
1.0	33.33333333333333
0.0	0.0
1.0	50.0
1.0	33.33333333333333
1.0	100.0
1.0	100.0
1.0	66.66666666666666



In [62]:
if False:
    import warnings
    warnings.filterwarnings('ignore')
    # Example usage
    # file_path = "full_logs_test_item_15_to_19_gpt_4_alone.txt"
    # file_path = "full_logs_test_item_20_39_claude_il.txt"
    # file_path = "full_logs_test_item_20_39_gpt_4_alone.txt"
    # file_path = "full_logs_test_item_30_40_gpt_4_il.txt"

    results = []
    full_logs = []
    results, full_logs = run_llm(llm_gpt3_5, start, end, expert_predictor=None)
    #results, full_logs = run_llm(llm_claude, 30, 40, expert_predictor=expert_predictor)

In [63]:
if False:
    display_result(results)

    file_path = f"full_logs_{start}_{end-1}_gpt3_5_{run}.txt"
    write_dict_to_file(full_logs, file_path)

================================== GPT-4 ==============================================

In [64]:
import warnings
warnings.filterwarnings('ignore')
# Example usage
# file_path = "full_logs_test_item_15_to_19_gpt_4_alone.txt"
# file_path = "full_logs_test_item_20_39_claude_il.txt"
# file_path = "full_logs_test_item_20_39_gpt_4_alone.txt"
# file_path = "full_logs_test_item_30_40_gpt_4_il.txt"

results = []
full_logs = []
results, full_logs = run_llm(llm_gpt4, start, end, expert_predictor=expert_predictor, top_k=1)
#results, full_logs = run_llm(llm_claude, 30, 40, expert_predictor=expert_predictor)


#### Goal 40: Instruction: i'm looking for a flats made of ethylene vinyl. choose the ones in tan color and size 9.5 narrow, and price lower than 80.00 dollars ####
loop number:1
Here's one suggestion for the command: search with 'tan ethylene vinyl flats'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I need to search for tan ethylene vinyl flats.",
        "reasoning": "The instruction asks for flats made of ethylene vinyl in tan color, size 9.5 narrow, and price lower than $80.",
        "plan": "- search for tan ethylene vinyl flats\n- find the right item\n- make the right purchase",
        "criticism": "N/A"
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input": "tan ethylene vinyl flats, size 9.5 narrow, under $80"
        }
    }
}
Command search returned: =Observation=
Instruction:
i'm looki

loop number:5
Here's one suggestion for the command: click with '< prev'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I should go back to the previous page to select the tan color.",
        "reasoning": "The features confirm that the material is ethylene vinyl acetate, but I need to select the tan color as per the instruction.",
        "plan": "- click '< Prev' to go back to the previous page\n- select the tan color\n- make the right purchase",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "< Prev"
        }
    }
}
Command click returned: =Observation=
You have clicked 9.5 narrow.
Instruction:
i'm looking for a flats made of ethylene vinyl. choose the ones in tan color and size 9.5 narrow, and price lower than 80.00 dollars
[button] Back to Search [button_]
[bu

loop number:2
Here's one suggestion for the command: click with 'next >'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I can only click 'Next >' to go to the next page, or click an item like B09QSVLR8X",
        "reasoning": "None of the items on this page match the criteria of winter warm ankle boots for women, size 7.5, and price lower than $50.",
        "plan": "- go to the next page\n- find the right product\n- make the purchase",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Next >"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for winter warm ankle boots for women. my size is 7.5, and price lower than 50.00 dollars
[button] Back to Search [button_]
Page 2 (Total results: 50)
[button] < Prev [button_]
[button] Next > [butt

{
    "thoughts": {
        "text": "I can only click 'Back to Search', '< Prev', 'Next >', or click an item like B09B1398P3",
        "reasoning": "None of the items on this page match the criteria of winter warm ankle boots for women, size 7.5, and price lower than $50.",
        "plan": "- go back to search\n- search again with a more specific query\n- find the right product\n- make the purchase",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Back to Search"
        }
    }
}
Command click returned: =Observation=
WebShop
Instruction: 
i am looking for winter warm ankle boots for women. my size is 7.5, and price lower than 50.00 dollars
[button] Search [button_]

loop number:6
Here's one suggestion for the command: search with 'winter warm ankle boots for women'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specifi

loop number:10
Here's one suggestion for the command: click with 'buy now'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I can click 'Description', 'Features', 'Reviews', or 'Buy Now'",
        "reasoning": "The selected item matches the criteria of winter warm ankle boots for women, size 7.5, and price lower than $50. I will click 'Buy Now' as suggested.",
        "plan": "- click Buy Now\n- complete the purchase",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Buy Now"
        }
    }
}
Command click returned: =Observation=
Thank you for shopping with us!
Your code: 
None
 (Paste it in your MTurk interface.)
Purchased
asin
B09MMYV8M8
options
{"color": "c02-blue", "size": "7.5"}
attrs
None
category
None
query
None
product category
None
Target
asin
options
attrs
pr

...

loop number:15
Here's one suggestion for the command: click with 'next >'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I need to find a 9 drawer dresser with a bronze finish.",
        "reasoning": "Clicking 'Next >' will take me to the next page with more options.",
        "plan": "- click on 'Next >' to go to the next page",
        "criticism": "Going to the next page might not guarantee finding the right item, but it's worth exploring more options."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Next >"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for a 9 drawer dresser with a bronze finish
[button] Back to Search [button_]
Page 2 (Total results: 50)
[button] < Prev [button_]
[button] Next > [button_]

[button] B08C9T6Z9M [button_]
Knocbel Vintage A

{
    "thoughts": {
        "text": "I still need to find a 9 drawer dresser with a bronze finish.",
        "reasoning": "Clicking '< Prev' will take me back to the previous page of search results.",
        "plan": "- click on '< Prev' to go back to the previous page",
        "criticism": "I haven't found a 9 drawer dresser with a bronze finish yet, but going back to the previous page might help me find one."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "< Prev"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for a 9 drawer dresser with a bronze finish
[button] Back to Search [button_]
Page 2 (Total results: 50)
[button] < Prev [button_]
[button] Next > [button_]

[button] B08C9T6Z9M [button_]
Knocbel Vintage Aesthetic 6-Drawer Chest with Bronze Finish Handles, Solid Wood Double Dresser Chest of Drawers, Fully Assembled, 55" L x 16" W x 34" H
$679.99

[button] B071HHBPQY [button_]
Progressive Furniture

loop number:3
Here's one suggestion for the command: click with 'red'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I should select the red color option for the shirt.",
        "reasoning": "The instruction specified a red shirt, so I need to choose the red color option.",
        "plan": "- select the red color\n- select the large size\n- make the purchase",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "red"
        }
    }
}
Command click returned: =Observation=
You have clicked red.
Instruction:
i'm looking for a men's red button down casual shirt that is a large slim fit, and price lower than 30.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
color
  [button] blue [button_]
  [clicked button] red [clicked button_]
  [button] yellow [but

{
    "thoughts": {
        "text": "I have successfully made the purchase.",
        "reasoning": "The purchase has been completed, and the task is finished.",
        "plan": "- finish the task",
        "criticism": "N/A"
    },
    "command": {
        "name": "finish",
        "args": {
            "response": "I have successfully purchased the men's red button down casual shirt that is a large slim fit, and the price is lower than $30.00. Thank you for shopping with us!"
        }
    }
}
#### Goal 48: Instruction: i want to find a teeth whitening device kit for sensitive teeth, and price lower than 90.00 dollars ####
loop number:1
Here's one suggestion for the command: search with 'teeth whitening device kit'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "There is only one search button, so I can only search",
        "reasoning": 

loop number:2
Here's one suggestion for the command: click with 'b01a99ge8i'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I need to click on the B01A99GE8I item to see if it matches the requirements.",
        "reasoning": "The item is kosher certified and Irish, but I need to check if it has a Halloween pattern and is under $40.",
        "plan": "- click on the item\n- check if it matches the requirements\n- make the purchase if it does",
        "criticism": "N/A"
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "B01A99GE8I"
        }
    }
}
Command click returned: =Observation=
Instruction:
i am looking for kosher certified irish fortune cookies with pattern name :halloween, and price lower than 40.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
pattern name
  [butt

In [65]:
display_result(results)
#file_path = "full_logs_0_19_gpt3_5.txt"
#file_path = "full_logs_20_29_gpt4_il.txt"

file_path = f"full_logs_{start}_{end-1}_gpt4_il_{run}.txt"
write_dict_to_file(full_logs, file_path)

purchase	reward
1.0	75.0
1.0	100.0
1.0	100.0
1.0	33.33333333333333
1.0	75.0
1.0	25.0
0.0	0.0
1.0	100.0
1.0	100.0
1.0	100.0



In [66]:
import warnings
warnings.filterwarnings('ignore')
# Example usage
# file_path = "full_logs_test_item_15_to_19_gpt_4_alone.txt"
# file_path = "full_logs_test_item_20_39_claude_il.txt"
# file_path = "full_logs_test_item_20_39_gpt_4_alone.txt"
# file_path = "full_logs_test_item_30_40_gpt_4_il.txt"

results = []
full_logs = []
results, full_logs = run_llm(llm_gpt4, start, end, expert_predictor=expert_predictor, top_k=5)
#results, full_logs = run_llm(llm_claude, 30, 40, expert_predictor=expert_predictor)


#### Goal 40: Instruction: i'm looking for a flats made of ethylene vinyl. choose the ones in tan color and size 9.5 narrow, and price lower than 80.00 dollars ####
loop number:1
Here's a few suggestions for the command: search with 'tan ethylene vinyl flats', search with 'tan ethylene vinyl flats size 9.5', search with 'tinted ethylene vinyl flats', search with 'tan ethylene vinyl flats 9.5 narrow', search with 'tan ethylene vinyl flats 9.5'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I need to search for tan ethylene vinyl flats in size 9.5 narrow and under $80.",
        "reasoning": "The instruction provided multiple search suggestions, but the most relevant one is 'tan ethylene vinyl flats 9.5 narrow' as it includes the color, material, size, and width.",
        "plan": "- search for 'tan ethylene vinyl flats 9.5 narrow'\n- find 

loop number:5
Here's a few suggestions for the command: click with 'b07myk47jv', click with 'next >', click with 'b08r2zf8qp', click with 'b09cynh7ls', click with 'next >'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I still need to find a tan ethylene vinyl flat in size 9.5 narrow and under $80.",
        "reasoning": "None of the options provided in the search results on this page mention ethylene vinyl material either. I should check the next page for more options.",
        "plan": "- click 'Next >'\n- find a suitable item under $80\n- make the purchase",
        "criticism": "There's a chance that the next page might not have the desired item either, but it's worth checking."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Next >"
        }
    }
}
Command click returned: =Observation=


loop number:10
Here's a few suggestions for the command: click with '< prev', click with 'next >', click with '< prev', click with '< prev', click with '< prev'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I need to find a tan ethylene vinyl flat in size 9.5 narrow and under $80.",
        "reasoning": "None of the options provided in the search results explicitly mention ethylene vinyl material. I should check the next page for more options.",
        "plan": "- click 'Next >'\n- find a suitable item under $80\n- make the purchase",
        "criticism": "There's a chance that the next page might not have the desired item either, but it's worth checking."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Next >"
        }
    }
}
Command click returned: =Observation=
Instruction:
i'm looking f

{
    "thoughts": {
        "text": "I need to find a tan ethylene vinyl flat in size 9.5 narrow and under $80.",
        "reasoning": "None of the options provided in the search results explicitly mention ethylene vinyl material. I should check the next page for more options.",
        "plan": "- click 'Next >'\n- find a suitable item under $80\n- make the purchase",
        "criticism": "There's a chance that the next page might not have the desired item either, but it's worth checking."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Next >"
        }
    }
}
Command click returned: =Observation=
Instruction:
i'm looking for a flats made of ethylene vinyl. choose the ones in tan color and size 9.5 narrow, and price lower than 80.00 dollars
[button] Back to Search [button_]
Page 7 (Total results: 50)
[button] < Prev [button_]
[button] Next > [button_]

loop number:15
Here's a few suggestions for the command: click with 'back to search', c

{
    "thoughts": {
        "text": "I need to find a tan ethylene vinyl flat in size 9.5 narrow and under $80.",
        "reasoning": "None of the options provided in the search results explicitly mention ethylene vinyl material. I should check the next page for more options.",
        "plan": "- click 'Next >'\n- find a suitable item under $80\n- make the purchase",
        "criticism": "There's a chance that the next page might not have the desired item either, but it's worth checking."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Next >"
        }
    }
}
Command click returned: =Observation=
Instruction:
i'm looking for a flats made of ethylene vinyl. choose the ones in tan color and size 9.5 narrow, and price lower than 80.00 dollars
[button] Back to Search [button_]
Page 2 (Total results: 50)
[button] < Prev [button_]
[button] Next > [button_]

[button] B07RB7S2D3 [button_]
Birkenstock Men's Milano Sling Back Sandals
$85.35 to $11

...

loop number:1
Here's a few suggestions for the command: search with 'mens red button down casual shirt', search with 'mens red button down casual shirt large slim fit', search with 'mens red button down casual shirt in large', search with 'mens button down casual shirt in large slim fit', search with 'mens button down casual shirt'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I need to search for a men's red button down casual shirt that is a large slim fit and priced lower than $30.",
        "reasoning": "There is only one search button, so I can only search.",
        "plan": "- search for the item\n- find the right product\n- make the purchase",
        "criticism": "N/A"
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input": "mens red button down casual shirt large slim fit under $30"
        

loop number:4
Here's a few suggestions for the command: click with 'b09plfj8y2', click with 'b09qqlm6j6', click with 'back to search', click with 'next >', click with '< prev'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I need to find a men's red button down casual shirt that is a large slim fit and priced lower than $30.",
        "reasoning": "I can only click on the product buttons, 'Back to Search', 'Next >', or '< Prev'.",
        "plan": "- click on the right product\n- check the details\n- make the purchase",
        "criticism": "None of the suggested products seem to match the criteria, so I should click 'Next >' to check more options."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Next >"
        }
    }
}
Command click returned: =Observation=
Instruction:
i'm looking for a men'

loop number:8
Here's a few suggestions for the command: click with 'b09p89cndd', click with 'b09p89cndd', click with 'back to search', click with 'b09p884b4k', click with 'next >'.
Please use this suggestion as a reference and make your own judgement. Determine which next command to use, and respond using the JSON format specified above:
{
    "thoughts": {
        "text": "I need to find a men's red button down casual shirt that is a large slim fit and priced lower than $30.",
        "reasoning": "I can only click on the product buttons or 'Back to Search' or 'Next >'.",
        "plan": "- click on the right product\n- check the details\n- make the purchase",
        "criticism": "None of the products on this page match the criteria, so I should click 'Next >' to check the next page."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "Next >"
        }
    }
}
Command click returned: =Observation=
Instruction:
i'm looking for a men's red but

{
    "thoughts": {
        "text": "I have successfully purchased the men's red button down casual shirt that is a large slim fit and priced lower than $30.",
        "reasoning": "I have completed the task and there are no more actions to take.",
        "plan": "- finish the task",
        "criticism": "N/A"
    },
    "command": {
        "name": "finish",
        "args": {
            "response": "I have successfully purchased the item as per your requirements. Thank you for shopping with us!"
        }
    }
}
#### Goal 48: Instruction: i want to find a teeth whitening device kit for sensitive teeth, and price lower than 90.00 dollars ####
loop number:1
Here's a few suggestions for the command: search with 'teeth whitening device kit', search with 'teeth whitening device for sensitive teeth', search with 'teeth whitening device', search with 'tooth whitening device for sensitive teeth', search with 'teeth whitening device kit sensitive'.
Please use this suggestion as a reference 

{
    "thoughts": {
        "text": "I need to search for kosher certified irish fortune cookies with a halloween pattern and a price lower than $40.",
        "reasoning": "The search query should include the main keywords from the instruction.",
        "plan": "- search for 'kosher certified irish fortune cookies halloween'\n- check the results and click on the relevant item\n- make the purchase if the price is lower than $40",
        "criticism": "I should make sure to include all important keywords in the search query."
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input": "kosher certified irish fortune cookies halloween"
        }
    }
}
Command search returned: =Observation=
Instruction:
i am looking for kosher certified irish fortune cookies with pattern name :halloween, and price lower than 40.00 dollars
[button] Back to Search [button_]
Page 1 (Total results: 50)
[button] Next > [button_]

[button] B01A99GE8I [button_]
Irish Fortune 

In [67]:
display_result(results)
file_path = f"full_logs_{start}_{end-1}_gpt4_il_top_5_{run}.txt"
write_dict_to_file(full_logs, file_path)

purchase	reward
0.0	0.0
1.0	100.0
1.0	50.0
1.0	33.33333333333333
1.0	50.0
1.0	50.0
0.0	0.0
1.0	100.0
1.0	100.0
1.0	100.0



In [68]:
if False:
    import warnings
    warnings.filterwarnings('ignore')
    # Example usage
    # file_path = "full_logs_test_item_15_to_19_gpt_4_alone.txt"
    # file_path = "full_logs_test_item_20_39_claude_il.txt"
    # file_path = "full_logs_test_item_20_39_gpt_4_alone.txt"
    # file_path = "full_logs_test_item_30_40_gpt_4_il.txt"

    results = []
    full_logs = []
    results, full_logs = run_llm(llm_gpt4, start, end, expert_predictor=None)
    #results, full_logs = run_llm(llm_claude, 30, 40, expert_predictor=expert_predictor)


In [69]:
if False:
    display_result(results)
    file_path = f"full_logs_{start}_{end-1}_gpt4_{run}.txt"
    write_dict_to_file(full_logs, file_path)

# Below is For Testing Purpose

In [24]:
#obs, info = env.reset(0)
#print(info['goal'])

Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


In [25]:
#agent = init_agent(ai_name, ai_role, tools, input_variables, llm, init_vectorstore())

#full_msg_log, actions = agent.run([f"I want you to help me find the right item and make the purchase in {LOOP_LIMIT} actions. " + info['goal']])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output i

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output i


> Finished chain.
{
    "thoughts": {
        "text": "I will click on the first item that meets all the requirements.",
        "reasoning": "The first item that meets all the requirements is the best option to purchase.",
        "plan": "- Click on the first item that meets all the requirements.",
        "criticism": "I need to ensure that the item meets all the requirements before clicking on it."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "b07fypsnh8"
        }
    }
}


> Entering new LLMChain chain...


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' i

Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output in the format as search[<query>].
  


> Finished chain.
{
    "thoughts": {
        "text": "I will purchase the item since it meets all the requirements and is within the price range.",
        "reasoning": "The item meets all the requirements and is within the price range, so it is the best option to purchase.",
        "plan": "- Purchase the item.",
        "criticism": "I need to ensure that the item is added to the cart and the purchase is successful."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "buy now"
        }
    }
}


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are Shopping AI, Web Shopper
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. I want you to help me find the right item and make the purchase in 20 actions. Instruction: i am looking for a gluten free, 100% vegan plant based protein shake that is soy-free, and price lower than 30.00 dollars


Constraints:
1. Your short term memory is short.
2. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. search: 
        Requirement:
        1. You can conduct search if your valid action has 'search[<QUERY>]'. 
        2. You need to come up with the best search query based on the instruction in the Observation.
        3. You need to check your past search queries to avoid repetitive searches.
        4. Try to relax constraint on the search query if you cannot find right items.
        Example:
        After you come up with the best query, output i


> Finished chain.
{
    "thoughts": {
        "text": "I will finish my shopping since I have successfully purchased the item.",
        "reasoning": "Since I have successfully purchased the item, there is no need to continue shopping.",
        "plan": "- Finish my shopping.",
        "criticism": "I need to ensure that the purchase is successful and the item is delivered."
    },
    "command": {
        "name": "finish",
        "args": {
            "response": "I have successfully purchased the gluten-free, 100% vegan plant-based protein shake that is soy-free and within the price range. Shopping complete."
        }
    }
}


In [33]:
results = []
result_01 = process_result(full_msg_log)
results.append(result_01)

In [31]:
#obs, info = env.reset(1)
#print(f"############### {info['goal']} ###############")
#agent = init_agent(ai_name, ai_role, tools, input_variables, llm, init_vectorstore(), verbose = False)

#full_msg_log, actions = agent.run([f"I want you to help me find the right item and make the purchase in {LOOP_LIMIT} actions. " + info['goal']])

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


################### Instruction: i need a 120 ml hair mask treatment, and price lower than 140.00 dollars ########################
{
    "thoughts": {
        "text": "I will start by searching for a 120 ml hair mask treatment with a price lower than 140.00 dollars.",
        "reasoning": "I will use the search command to find the hair mask treatment. I will also keep track of my past search queries to avoid repetitive searches.",
        "plan": "- Use the search command to find a 120 ml hair mask treatment with a price lower than 140.00 dollars.\n- Check my past search queries to avoid repetitive searches.\n- If I cannot find the right item, I will relax the constraint on the search query.\n- If I am running out of actions, I will make the purchase.",
        "criticism": "I need to be efficient with my search queries to avoid wasting actions."
    },
    "command": {
        "name": "search",
        "args": {
            "tool_input": "120 ml hair mask treatment price lower than 14

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


{
    "thoughts": {
        "text": "I will click on the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "reasoning": "The Linange Milk and Keratin Mask 500ml hair mask treatment is the only item on the page that meets the requirements of being a 120 ml hair mask treatment with a price lower than 140.00 dollars.",
        "plan": "- Click on the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "criticism": "I need to be careful to ensure that I am clicking on the correct item."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "B079HGJ5MH"
        }
    }
}


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


I will click on the 'Buy Now' button to purchase the Linange Milk and Keratin Mask 500ml hair mask treatment.

{
    "thoughts": {
        "text": "I will click on the 'Buy Now' button to purchase the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "reasoning": "The 'Buy Now' button is the next logical step to purchase the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "plan": "- Click on the 'Buy Now' button to purchase the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "criticism": "I need to ensure that I am clicking on the correct button to make the purchase."
    },
    "command": {
        "name": "click",
        "args": {
            "tool_input": "buy now"
        }
    }
}


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


The purchase has been made, so I will use the 'finish' command to signal that I have completed all my objectives.

{
    "thoughts": {
        "text": "I have completed the purchase of the Linange Milk and Keratin Mask 500ml hair mask treatment.",
        "reasoning": "Since the purchase has been made, there are no further actions required.",
        "plan": "- Use the 'finish' command to signal that I have completed all my objectives.",
        "criticism": "I need to ensure that I have completed all my objectives before using the 'finish' command."
    },
    "command": {
        "name": "finish",
        "args": {
            "response": "I have completed the purchase of the Linange Milk and Keratin Mask 500ml hair mask treatment."
        }
    }
}


In [38]:
print(full_msg_log)

[HumanMessage(content='Determine which next command to use, and respond using the format specified above:', additional_kwargs={}, example=False), AIMessage(content='{\n    "thoughts": {\n        "text": "I will start by searching for orange cargo pants that are suitable for everyday casual wear and hiking, and cost less than $30.",\n        "reasoning": "I need to find the right item that meets all the requirements specified by the user.",\n        "plan": "- Search for \'orange cargo pants for everyday casual wear and hiking\'\\n- Check the search results for the right item\\n- Click on the item to view more details\\n- If the item meets all the requirements, click on \'buy now\'\\n- If not, click on \'back to search\' and repeat the process",\n        "criticism": "I need to be careful not to waste too many actions on repetitive searches or irrelevant items."\n    },\n    "command": {\n        "name": "search",\n        "args": {\n            "tool_input": "orange cargo pants for eve

In [1]:
from output_parser import AutoGPTOutputParser
msg = """
I have successfully purchased the hair mask. I will now use the finish command to signal that I have completed all my objectives.

{
    "thoughts": {
        "text": "I have successfully purchased the hair mask. I will now use the finish command to signal that I have completed all my objectives.",
        "reasoning": "I have found the hair mask with the right price and size and have successfully purchased it. I will now use the finish command to signal that I have completed all my objectives.",
        "plan": "- Use the finish command to signal that I have completed all my objectives.",
        "criticism": "None"
    },
    "command": {
        "name": "finish",
        "args": {
            "response": "I have successfully purchased the hair mask and completed all my objectives."
        }
    }
}
"""
output_parser = AutoGPTOutputParser()
output_parser.parse(msg)

AutoGPTAction(name='finish', args={'response': 'I have successfully purchased the hair mask and completed all my objectives.'})

In [28]:
obs, info = env.reset(0)
action = "search[long clip-in hair extension natural looking]"
obs, reward, done, info = env.step(action)
info = convert_info(info)
print(info)



{'valid': ['click[back to search]', 'click[next >]', 'click[b09mm9ncqq]', 'click[b09g2f3hq2]', 'click[b09cgnh5xt]', 'click[b09qmj5kqh]', 'click[b09llrswf4]', 'click[b004rk7i2w]', 'click[b096mm7xm4]', 'click[b07cktxs1j]', 'click[b09s2z2gmj]', 'click[b08zkdqpz3]'], 'goal': 'Instruction: i need a long clip-in hair extension which is natural looking, and price lower than 30.00 dollars', 'score': 0.0, 'estimate_score': 0.0, 'prev_ob': 'WebShop\nInstruction: \ni need a long clip-in hair extension which is natural looking, and price lower than 30.00 dollars\n[button] Search [button_]\n', 'desc': '', 'feat': ''}


/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


In [20]:
action = "search[long clip-in hair extension natural looking]"
obs, reward, done, info = env.step(action)
print(obs)
print(info)

Instruction:
i need a long clip-in hair extension which is natural looking, and price lower than 30.00 dollars
[button] Back to Search [button_]
Page 1 (Total results: 50)
[button] Next > [button_]

[button] B09MM9NCQQ [button_]
Simayixx Christmas Gift Wigs Hair Extensions 24 Inches Synthetic Long Natural Black Thick Clip Straight Hair Stylish Fasionable Fabulous Hair Piece 6 Pieces Popular Hairstyle(E)
$7.83

[button] B09G2F3HQ2 [button_]
Lurrose 6pcs Girls Hair Bow Ribbon Barrettes Hair Bow Ribbon Clip Wig Hair Bow Clips with Alligator Clip and Hair Extensions Hair Accessory for Toddler Girls Children
$11.99

[button] B09CGNH5XT [button_]
Dual Magnetic Lashes, Magnets False Eyelashes, Soft 3D No Glue Fake Lashes Extension with Tweezers, Natural Look Eyelashes Set with 6 Pieces / 3 Pairs /Mini HD Makeup Mirror
$9.99

[button] B09QMJ5KQH [button_]
Messy Bun Scrunchie Hair Bun Extensions Wavy Curly Messy Donut Hairpieces Synthetic Donut Updo Hair Pieces for Women and Girls (black brown)

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(


In [21]:
action = "click[item - simayixx christmas gift wigs hair extensions 24 inches synthetic long natural black thick clip straight hair stylish fasionable fabulous hair piece 6 pieces popular hairstyle(e)]"
obs, reward, done, info = env.step(action)
print(obs)
print(info)
print(type(info))

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)


Instruction:
i need a long clip-in hair extension which is natural looking, and price lower than 30.00 dollars
[button] Back to Search [button_]
[button] < Prev [button_]
color
  [button] a [button_]
  [button] b [button_]
  [button] c [button_]
  [button] d [button_]
  [button] e [button_]
  [button] f [button_]
  [button] g [button_]
  [button] h [button_]
  [button] i [button_]
  [button] j [button_]
Simayixx Christmas Gift Wigs Hair Extensions 24 Inches Synthetic Long Natural Black Thick Clip Straight Hair Stylish Fasionable Fabulous Hair Piece 6 Pieces Popular Hairstyle(E)
Price: $7.83
Rating: N.A.
[button] Description [button_]
[button] Features [button_]
[button] Reviews [button_]
[button] Buy Now [button_]

{'valid': ['click[back to search]', 'click[< prev]', 'click[description]', 'click[features]', 'click[reviews]', 'click[buy now]', 'click[a]', 'click[b]', 'click[c]', 'click[d]', 'click[e]', 'click[f]', 'click[g]', 'click[h]', 'click[i]', 'click[j]'], 'goal': 'Instruction: i 

/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  warnings.warn(
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/flask/testing.py:71: DeprecationWarning: 'werkzeug.urls.url_parse' is deprecated and will be removed in Werkzeug 2.4. Use 'urllib.parse.urlsplit' instead.
  url = url_parse(path)
/home/yh/anaconda3/envs/webshop/lib/python3.8/site-packages/bs4/element.py:784: DeprecationWarning: The 'text' argument to find(